### 2 Обработка текста

В предыдущей главе мы углубились в структуру больших языковых моделей (LLM) и узнали, что они предварительно обучаются на огромных объемах текста, обрабатывая
по одному слову (токену) за раз. 

![](images/llm2.1.png)


### 2.1 Векторное представление (vector embedding)

Нейросетевые модели, не могут напрямую обрабатывать текст. Текст как тип данных несовместим с тензорными операциями. Следовательно, нужен способ представлять слова в виде векторов с непрерывными значениями. (ссылка узнать больше в Приложении A, раздел A2.2 «Понимание тензоров».)

Построение векторного представления может быть проведено разными способами (различные модели представления). 
Прочие типы данных также требуют преобразования в тензорный формат.  

![](images/llm2.2.png)

Существует несколько алгоритмов для создания представлений слов. Одним из первых и наиболее популярных примеров является Word2Vec. Основная идея заключается в том, что слова, встречающиеся в схожем контексте, имеют схожее значение. Следовательно, при проецировании в двумерное пространство, можно увидеть, что схожие термины группируются вместе.

![](images/llm2.3.png)

Помимо моделей представления слов, существуют модели представления предложений, абзацев, целых документов.
Таким образом, картинка из предыдущей главы (predicting next world) не вполне корректна. 

Хотя мы можем использовать существующие модели, LLM обычно создают свои собственные представления, которые являются частью входного слоя и обновляются во время обучения. Преимущество такого подхода - оптимизация для конкретной задачи и имеющихся данных. 

Размерность представления (двумерное на рисунке выше) может быть произвольной. Большая размерность может отражать более тонкие отношения, но за счет вычислительной эффективности. Эта размерность называется размерностью скрытых состояний модели (dimensionality of the model's hidden states). 
Эта размерность - компромисс между производительностью и эффективностью. 
GPT-2 (125M параметров) используют размерность 768. 
GPT-3 (175B параметров) использует размерность 12 288.

### 2.2 Токенизация текста

![](images/llm2.4.png)

Текст, который мы будем токенизировать для обучения LLM, представляет собой рассказ Эдит Уортон под названием «Вердикт». Текст доступен в Wikisource по адресу https://en.wikisource.org/wiki/The_Verdict 


In [61]:
# Чтение короткого рассказа в виде образца текста на Python

with open("source/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Как нам лучше всего разделить этот текст, чтобы получить список токенов? 

Используя простой пример текста, мы можем использовать команду re.split со следующим синтаксисом, чтобы разделить текст на пробельные символы:

In [62]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']



Изменим разделение регулярных выражений на пробелы (\s), запятые и точки ([,.]):

In [63]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


Пробелы и прочие специальные символы. При разработке простого токенизатора, следует ли нам кодировать пробелы как отдельные символы или просто удалять их, зависит от нашего приложения и его требований. Удаление пробелов снижает требования к памяти и вычислительным ресурсам. Однако сохранение пробелов может быть полезно, если мы обучаем модели, чувствительные к точной структуре текста (например, код Python, чувствительный к отступам и интервалам). Здесь мы удаляем пробелы для простоты и краткости токенизированных результатов. Позже мы переключимся на схему токенизации, включающую пробелы.

Изменим код, чтобы он мог обрабатывать и другие типы пунктуации, такие как вопросительные знаки, кавычки и двойные тире, которые мы видели ранее в первых 100 символах рассказа Эдит Уортон, а также дополнительные специальные символы.

In [64]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


Наша модель токенизации теперь может успешно обрабатывать различные специальные символы в тексте.

![](images/llm2.5.png)

Теперь, когда у нас есть работающий базовый токенизатор, применим его ко всему рассказу Эдит Уортон:

In [65]:
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])

4649
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### 2.3 Преобразование токенов в идентификаторы

Это преобразование является промежуточным шагом перед преобразованием в векторное представление.

Чтобы сопоставить ранее сгенерированные токены с идентификаторами, нужно создать словарь. Этот словарь определяет, как мы сопоставляем каждое уникальное слово и специальный символ с уникальным целым числом, как показано на рисунке. Мы создаем словарь, разбивая весь текст обучающего набора данных на отдельные токены. Эти отдельные токены затем сортируются в алфавитном порядке, а повторяющиеся токены удаляются. Затем уникальные токены объединяются в словарь, который определяет сопоставление каждого уникального токена с уникальным целочисленным значением. 

![](images/llm2.6.png)

In [66]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(vocab_size)

vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

1159
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Carlo;', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('Florence', 35)
('For', 36)
('Gallery', 37)
('Gideon', 38)
('Gisburn', 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
('Grindle:', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)
('Her', 51)


Как мы видим, словарь содержит отдельные токены, связанные с уникальными целочисленными метками.
Наша следующая цель — применить этот словарь для преобразования нового текста в идентификаторы токенов, как показано на рисунке.

![](images/llm2.7.png)


Далее, когда мы захотим преобразовать выходные данные LLM из чисел обратно в текст, нам также понадобится способ превратить идентификаторы в текст. Для этого мы можем создать обратную версию словаря, которая сопоставляет идентификаторы с соответствующими текстовыми токенами.

Реализуем полный класс токенизатора и детокенизатора.

In [67]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids]) 
        
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    
tokenizer = SimpleTokenizerV1(vocab)
 
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)

print(ids)
print(tokenizer.decode(ids))

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 773, 812, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.



### 2.4 Добавление специальных токенов контекста

Изменим токенизатор для обработки неизвестных слов. Также обсудим использование и добавление специальных токенов контекста, которые могут улучшить понимание моделью контекста или другой соответствующей информации в тексте. Эти специальные токены могут включать, например, маркеры для неизвестных слов и границ документа.

В частности, мы изменим словарь и токенизатор для поддержки двух новых токенов: <|unk|> и <|endoftext|>

![](images/llm2.9.png)

Как показано на рисунке, мы можем изменить токенизатор, чтобы он использовал токен <|unk|>, если он встретит слово, которое не является частью словаря. Кроме того, мы добавляем токен между несвязанными текстами. Например, при обучении GPT-подобных LLM на нескольких независимых документах или книгах обычно вставляется токен перед каждым документом или книгой, следующим за предыдущим источником текста Это помогает LLM понять, что, хотя эти текстовые источники объединены для обучения, на самом деле они не связаны друг с другом.

![](images/llm2.10.png)

In [68]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
 
print(len(vocab.items()))

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]
 
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
 
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

1161
('younger', 1156)
('your', 1157)
('yourself', 1158)
('<|endoftext|>', 1159)
('<|unk|>', 1160)
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
[1160, 5, 362, 1155, 642, 1000, 10, 1159, 57, 1013, 981, 1009, 738, 1013, 1160, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


Выше мы видим, что идентификатор 1159 соответствует токену-разделителю <|endoftext|>, а идентификатор 1160 используются для неизвестных слов.

Сравнивая приведенный выше детокенизированный текст с исходным входным текстом, мы знаем, что набор обучающих данных, рассказ Эдит Уортон «Вердикт», не содержал слов «Привет» и «дворец».

Также рассматривают дополнительные специальные токены, такие как следующие:

[BOS] (начало последовательности): этот токен отмечает начало текста.   
[EOS] (конец последовательности): этот токен располагается в конце текста. Например, при объединении двух разных статей или книг Википедии токен [EOS] указывает, где заканчивается одна статья и начинается следующая.  
[PAD] (заполнение): при обучении LLM с размерами пакетов больше единицы пакет может содержать тексты различной длины. Чтобы гарантировать одинаковую длину всех текстов, более короткие тексты расширяются или «дополняются» с помощью токена [PAD] до длины самого длинного текста в пакете.  

### 2.5 Кодирование пар байтов (Byte pair encoding)

В этом разделе рассматривается более сложная схема токенизации, основанная на концепции, называемой кодированием пар байтов (BPE). Такой токенизатор использовался для обучения LLM, таких как GPT-2, GPT-3 и исходной модели, используемой в ChatGPT.

Поскольку реализация BPE может быть сложной, мы будем использовать существующую библиотеку Python с открытым исходным кодом под названием tiktoken (https://github.com/openai/tiktoken), которая очень эффективно реализует алгоритм BPE на основе исходного кода на Rust. Как и другие библиотеки Python, мы можем установить библиотеку tiktoken через установщик pip Python из терминала.

In [69]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

tiktoken version: 0.6.0
[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


На основе идентификаторов токенов и декодированного текста выше мы можем сделать два примечательных наблюдения. Во-первых, токену <|endoftext|> присваивается относительно большой идентификатор токена, а именно 50256. Фактически, токенизатор BPE, который использовался для обучения таких моделей, как GPT-2, GPT-3, и исходная модель, используемая в ChatGPT имеет общий размер словаря 50 257, причем <|endoftext|> присвоен самый большой идентификатор токена.

Во-вторых, токенизатор BPE, указанный выше, правильно кодирует и декодирует неизвестные слова, такие как «someunknownPlace». Токенизатор BPE может обрабатывать любое неизвестное слово. Как это достигается без использования токенов <|unk|>?

Алгоритм, лежащий в основе BPE, разбивает слова, которых нет в его предопределенном словаре, на более мелкие подслова или даже отдельные символы, что позволяет ему обрабатывать слова, которых нет в словаре. Таким образом, благодаря алгоритму BPE, если токенизатор встречает незнакомое слово во время токенизации, он может представить его как последовательность токенов или символов подслова, как показано на рисунке.

![](images/llm2.11.png)

Подробное обсуждение и реализация BPE выходит за рамки этой книги, но, если коротко, словарь строится путем итеративного объединения часто встречающихся символов в подслова, а часто встречающихся подслов — в слова. Например, BPE начинает с добавления в свой словарь всех отдельных одиночных символов («a», «b», ...). На следующем этапе он объединяет комбинации символов, которые часто встречаются вместе, в подслова. Например, «d» и «e» могут быть объединены в подслово «de», которое часто встречается во многих английских словах, таких как «define», «dependent», «made» и «hidden».

### 2.6 Выборка данных с помощью скользящего окна

Следующим шагом является создание пар вход-выход, необходимых для обучения LLM.
Как выглядят эти пары вход-выход? 

![](images/llm2.12.png)

Реализуем теперь загрузчик данных, который извлекает пары вход-вызод из набора обучающих данных, используя подход скользящего окна.

Для чего токенизируем весь рассказ The Verdict, с которым мы работали ранее, с помощью токенизатора BPE


In [70]:
with open("source/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
 
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


Один из самых простых и интуитивно понятных способов создания пар входных данных для задачи прогнозирования следующего слова — это создание двух переменных, x и y, где x содержит входные токены, а y содержит целевые значения, которые представляют собой входные данные, сдвинутые на 1:

In [71]:
enc_sample = enc_text[50:]
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


Обрабатывая входные данные вместе с целевыми объектами, которые представляют собой входные данные, сдвинутые на одну позицию, мы можем затем создать задачи прогнозирования следующего слова, следующим образом:

In [72]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


Все, что находится слева от стрелки (---->), относится к входным данным, которые LLM получит, а идентификатор токена в правой части стрелки представляет собой целевой идентификатор токена, который LLM должен предсказать.

В целях иллюстрации давайте повторим предыдущий код, но преобразуем идентификаторы токенов в текст.

In [73]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


Теперь мы создали пары вход-цель, которые можно использовать для обучения LLM в следующих главах.

Прежде чем мы сможем превратить токены во встраивания, есть еще только одна задача, как мы упоминали в начале этой главы: реализация эффективного загрузчика данных, который выполняет итерацию по входному набору данных и возвращает входные и целевые значения в виде тензоров PyTorch, что можно представить себе как как многомерные массивы.

В частности, мы заинтересованы в возврате двух тензоров: входного, содержащего текст, который видит LLM, и целевого тензора, который включает в себя цели, которые LLM должен предсказать, как показано на рисунке 2.13.

Рисунок 2.13. Чтобы реализовать эффективные загрузчики данных, мы собираем входные данные в тензор x, где каждая строка представляет один входной контекст. Второй тензор y содержит соответствующие цели прогнозирования (следующие слова), которые создаются путем сдвига входных данных на одну позицию.

![](images/llm2.13.png)

Хотя на рисунке 2.13 для иллюстрации токены показаны в строковом формате, реализация кода будет работать непосредственно с идентификаторами токенов, поскольку метод кодирования токенизатора BPE выполняет как токенизацию, так и преобразование в идентификаторы токенов за один шаг.

Для эффективной реализации загрузчика данных мы будем использовать встроенные классы PyTorch Dataset и DataLoader. Дополнительную информацию и рекомендации по установке PyTorch см. в разделе A.1.3 «Установка PyTorch» в Приложении A.

Код класса набора данных показан в листинге 2.5:

Листинг 2.5. Набор данных для пакетных входных и целевых значений

In [74]:
import torch
from torch.utils.data import Dataset, DataLoader
 
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
 
        token_ids = tokenizer.encode(txt)
 
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
 
    def __len__(self):
        return len(self.input_ids)
 
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

Класс GPTDatasetV1 в листинге 2.5 основан на классе набора данных PyTorch и определяет, как отдельные строки извлекаются из набора данных, где каждая строка состоит из нескольких идентификаторов токенов (на основе max_length), назначенных тензору input_chunk. Тензор target_chunk содержит соответствующие цели. Я рекомендую прочитать дальше, чтобы увидеть, как выглядят данные, возвращаемые из этого набора данных, когда мы объединяем набор данных с PyTorch DataLoader — это принесет дополнительную интуицию и ясность.

Если вы впервые знакомы со структурой классов набора данных PyTorch, например, показанной в листинге 2.5, прочтите раздел A.6 «Настройка эффективных загрузчиков данных» в приложении A, в котором объясняется общая структура и использование классов PyTorch Dataset и DataLoader.

Следующий код будет использовать GPTDatasetV1 для пакетной загрузки входных данных через PyTorch DataLoader:

Листинг 2.6. Загрузчик данных для генерации пакетов с парами вход-с

In [75]:
def create_dataloader_v1(txt, batch_size=4, 
        max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

Давайте протестируем загрузчик данных с размером пакета 1 для LLM с размером контекста 4, чтобы понять, как класс GPTDatasetV1 из листинга 2.5 и функция create_dataloader_v1 из листинга 2.6 работают вместе:

In [76]:
with open("source/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
 
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


/tmp/ipykernel_202984/3364176621.py:15: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  self.input_ids.append(torch.tensor(input_chunk))


Переменная first_batch содержит два тензора: первый тензор хранит идентификаторы входных токенов, а второй тензор хранит идентификаторы целевых токенов. Поскольку max_length установлено равным 4, каждый из двух тензоров содержит 4 идентификатора токена. Обратите внимание, что входной размер 4 относительно мал и выбран только для иллюстрации. Обычно LLM обучают с размером входных данных не менее 256.

Чтобы проиллюстрировать значение шага=1, давайте возьмем еще один пакет из этого набора данных:

In [77]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


Если мы сравним первый и второй пакет, мы увидим, что идентификаторы токенов второго пакета сдвинуты на одну позицию по сравнению с первым пакетом (например, второй идентификатор во входных данных первого пакета равен 367, что является первым идентификатором вход второго пакета). Настройка шага определяет количество позиций, на которые смещаются входные данные между пакетами, имитируя подход скользящего окна, как показано на рисунке 2.14.

Рисунок 2.14. При создании нескольких пакетов из входного набора данных мы перемещаем окно ввода по тексту. Если шаг установлен на 1, мы сдвигаем окно ввода на 1 позицию при создании следующего пакета. Если мы установим шаг равным размеру окна ввода, мы сможем предотвратить перекрытие между пакетами.

![](images/llm2.14.png)


Упражнение 2.2. Загрузчики данных с разными шагами и размерами контекста.

Чтобы лучше понять, как работает загрузчик данных, попробуйте запустить его с различными настройками, такими как max_length=2 и шаг=2 и max_length=8 и шаг=2.




Размеры пакетов, равные 1, например, которые мы до сих пор выбирали из загрузчика данных, полезны для иллюстративных целей. Если у вас есть предыдущий опыт глубокого обучения, вы, возможно, знаете, что небольшие размеры пакетов требуют меньше памяти во время обучения, но приводят к более шумным обновлениям модели. Как и в обычном глубоком обучении, размер пакета — это компромисс и гиперпараметр, с которым можно экспериментировать при обучении LLM.

Прежде чем мы перейдем к двум последним разделам этой главы, посвященным созданию векторов внедрения из идентификаторов токенов, давайте кратко рассмотрим, как мы можем использовать загрузчик данных для выборки с размером пакета больше 1:

In [78]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)
 
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[ 5032,    11,   339,  5257],
        [   11,  3595,   520,  5493],
        [  286, 18113,   544,   338],
        [  373,   866,   287,  6245],
        [  340,   373,   262,   938],
        [ 6687,   284,  2107,  1231],
        [  314,  3947,   284,   766],
        [  257,   410,  5040,   329]])

Targets:
 tensor([[   11,   339,  5257,   284],
        [ 3595,   520,  5493,     0],
        [18113,   544,   338, 10953],
        [  866,   287,  6245,   684],
        [  373,   262,   938,  1517],
        [  284,  2107,  1231,   326],
        [ 3947,   284,   766,   257],
        [  410,  5040,   329,   257]])


Обратите внимание, что мы увеличиваем шаг до 4. Это необходимо для полного использования набора данных (мы не пропускаем ни одного слова), но также для избежания любого перекрытия между пакетами, поскольку большее перекрытие может привести к увеличению переобучения.

В последних двух разделах этой главы мы реализуем уровни внедрения, которые преобразуют идентификаторы токенов в непрерывные векторные представления, которые служат форматом входных данных для LLM.


Next, we remove the first 50 tokens from the dataset for demonstration purposes as it results in a slightly more interesting text passage in the next steps:

enc_sample = enc_text[50:]

One of the easiest and most intuitive ways to create the input-target pairs for the next-word prediction task is to create two variables, x and y, where x contains the input tokens and y contains the targets, which are the inputs shifted by 1:

context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

Running the above code prints the following output:

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]

Processing the inputs along with the targets, which are the inputs shifted by one position, we can then create the next-word prediction tasks depicted earlier in figure 2.12, as follows:

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

The code above prints the following:

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257



Everything left of the arrow (---->) refers to the input an LLM would receive, and the token ID on the right side of the arrow represents the target token ID that the LLM is supposed to predict.

For illustration purposes, let's repeat the previous code but convert the token IDs into text:


for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))


The following outputs show how the input and outputs look in text format:

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a



We've now created the input-target pairs that we can turn into use for the LLM training in upcoming chapters.

There's only one more task before we can turn the tokens into embeddings, as we mentioned at the beginning of this chapter: implementing an efficient data loader that iterates over the input dataset and returns the inputs and targets as PyTorch tensors, which can be thought of as multidimensional arrays.

In particular, we are interested in returning two tensors: an input tensor containing the text that the LLM sees and a target tensor that includes the targets for the LLM to predict, as depicted in Figure 2.13.

Figure 2.13 To implement efficient data loaders, we collect the inputs in a tensor, x, where each row represents one input context. A second tensor, y, contains the corresponding prediction targets (next words), which are created by shifting the input by one position.




While Figure 2.13 shows the tokens in string format for illustration purposes, the code implementation will operate on token IDs directly since the encode method of the BPE tokenizer performs both tokenization and conversion into token IDs as a single step.

For the efficient data loader implementation, we will use PyTorch's built-in Dataset and DataLoader classes. For additional information and guidance on installing PyTorch, please see section A.1.3, Installing PyTorch, in Appendix A.

The code for the dataset class is shown in code listing 2.5:

Listing 2.5 A dataset for batched inputs and targets

import torch
from torch.utils.data import Dataset, DataLoader
 
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
 
        token_ids = tokenizer.encode(txt)
 
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
 
    def __len__(self):
        return len(self.input_ids)
 
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]




The GPTDatasetV1 class in listing 2.5 is based on the PyTorch Dataset class and defines how individual rows are fetched from the dataset, where each row consists of a number of token IDs (based on a max_length) assigned to an input_chunk tensor. The target_chunk tensor contains the corresponding targets. I recommend reading on to see how the data returned from this dataset looks like when we combine the dataset with a PyTorch DataLoader -- this will bring additional intuition and clarity.

If you are new to the structure of PyTorch Dataset classes, such as shown in listing 2.5, please read section A.6, Setting up efficient data loaders, in Appendix A, which explains the general structure and usage of PyTorch Dataset and DataLoader classes.

The following code will use the GPTDatasetV1 to load the inputs in batches via a PyTorch DataLoader:

Listing 2.6 A data loader to generate batches with input-with pairs

def create_dataloader_v1(txt, batch_size=4, 
        max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4 to develop an intuition of how the GPTDatasetV1 class from listing 2.5 and the create_dataloader_v1 function from listing 2.6 work together:

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
 
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

Executing the preceding code prints the following:

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]




The first_batch variable contains two tensors: the first tensor stores the input token IDs, and the second tensor stores the target token IDs. Since the max_length is set to 4, each of the two tensors contains 4 token IDs. Note that an input size of 4 is relatively small and only chosen for illustration purposes. It is common to train LLMs with input sizes of at least 256.

To illustrate the meaning of stride=1, let's fetch another batch from this dataset:

second_batch = next(data_iter)
print(second_batch)


The second batch has the following contents:

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]

If we compare the first with the second batch, we can see that the second batch's token IDs are shifted by one position compared to the first batch (for example, the second ID in the first batch's input is 367, which is the first ID of the second batch's input). The stride setting dictates the number of positions the inputs shift across batches, emulating a sliding window approach, as demonstrated in Figure 2.14.

Figure 2.14 When creating multiple batches from the input dataset, we slide an input window across the text. If the stride is set to 1, we shift the input window by 1 position when creating the next batch. If we set the stride equal to the input window size, we can prevent overlaps between the batches.


Exercise 2.2 Data loaders with different strides and context sizes

To develop more intuition for how the data loader works, try to run it with different settings such as max_length=2 and stride=2 and max_length=8 and stride=2.




Batch sizes of 1, such as we have sampled from the data loader so far, are useful for illustration purposes. If you have previous experience with deep learning, you may know that small batch sizes require less memory during training but lead to more noisy model updates. Just like in regular deep learning, the batch size is a trade-off and hyperparameter to experiment with when training LLMs.

Before we move on to the two final sections of this chapter that are focused on creating the embedding vectors from the token IDs, let's have a brief look at how we can use the data loader to sample with a batch size greater than 1:


dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)
 
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)


This prints the following:

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
 
Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])




Note that we increase the stride to 4. This is to utilize the data set fully (we don't skip a single word) but also avoid any overlap between the batches, since more overlap could lead to increased overfitting.

In the final two sections of this chapter, we will implement embedding layers that convert the token IDs into continuous vector representations, which serve as input data format for LLMs.



2.7 Creating token embeddings

The last step for preparing the input text for LLM training is to convert the token IDs into embedding vectors, as illustrated in Figure 2.15, which will be the focus of these two last remaining sections of this chapter.

Figure 2.15 Preparing the input text for an LLM involves tokenizing text, converting text tokens to token IDs, and converting token IDs into vector embedding vectors. In this section, we consider the token IDs created in previous sections to create the token embedding vectors.



In addition to the processes outlined in Figure 2.15, it is important to note that we initialize these embedding weights with random values as a preliminary step. This initialization serves as the starting point for the LLM's learning process. We will optimize the embedding weights as part of the LLM training in chapter 5.

A continuous vector representation, or embedding, is necessary since GPT-like LLMs are deep neural networks trained with the backpropagation algorithm. If you are unfamiliar with how neural networks are trained with backpropagation, please read section A.4, Automatic differentiation made easy, in Appendix A.

Let's illustrate how the token ID to embedding vector conversion works with a hands-on example. Suppose we have the following four input tokens with IDs 2, 3, 5, and 1:

input_ids = torch.tensor([2, 3, 5, 1])

For the sake of simplicity and illustration purposes, suppose we have a small vocabulary of only 6 words (instead of the 50,257 words in the BPE tokenizer vocabulary), and we want to create embeddings of size 3 (in GPT-3, the embedding size is 12,288 dimensions):

vocab_size = 6
output_dim = 3

Using the vocab_size and output_dim, we can instantiate an embedding layer in PyTorch, setting the random seed to 123 for reproducibility purposes:

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

The print statement in the preceding code example prints the embedding layer's underlying weight matrix:

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)



We can see that the weight matrix of the embedding layer contains small, random values. These values are optimized during LLM training as part of the LLM optimization itself, as we will see in upcoming chapters. Moreover, we can see that the weight matrix has six rows and three columns. There is one row for each of the six possible tokens in the vocabulary. And there is one column for each of the three embedding dimensions.

After we instantiated the embedding layer, let's now apply it to a token ID to obtain the embedding vector:

print(embedding_layer(torch.tensor([3])))

The returned embedding vector is as follows:

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)

If we compare the embedding vector for token ID 3 to the previous embedding matrix, we see that it is identical to the 4th row (Python starts with a zero index, so it's the row corresponding to index 3). In other words, the embedding layer is essentially a look-up operation that retrieves rows from the embedding layer's weight matrix via a token ID.


Embedding layers versus matrix multiplication

For those who are familiar with one-hot encoding, the embedding layer approach above is essentially just a more efficient way of implementing one-hot encoding followed by matrix multiplication in a fully connected layer, which is illustrated in the supplementary code on GitHub at https://github.com/rasbt/LLMs-from-scratch/tree/main/ch02/03_bonus_embedding-vs-matmul. Because the embedding layer is just a more efficient implementation equivalent to the one-hot encoding and matrix-multiplication approach, it can be seen as a neural network layer that can be optimized via backpropagation.

Previously, we have seen how to convert a single token ID into a three-dimensional embedding vector. Let's now apply that to all four input IDs we defined earlier (torch.tensor([2, 3, 5, 1])):

print(embedding_layer(input_ids))

The print output reveals that this results in a 4x3 matrix:

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)

Each row in this output matrix is obtained via a lookup operation from the embedding weight matrix, as illustrated in Figure 2.16.

Figure 2.16 Embedding layers perform a look-up operation, retrieving the embedding vector corresponding to the token ID from the embedding layer's weight matrix. For instance, the embedding vector of the token ID 5 is the sixth row of the embedding layer weight matrix (it is the sixth instead of the fifth row because Python starts counting at 0). For illustration purposes, we assume that the token IDs were produced by the small vocabulary we used in section 2.3.

This section covered how we create embedding vectors from token IDs. The next and final section of this chapter will add a small modification to these embedding vectors to encode positional information about a token within a text.


2.8 Encoding word positions

In the previous section, we converted the token IDs into a continuous vector representation, the so-called token embeddings. In principle, this is a suitable input for an LLM. However, a minor shortcoming of LLMs is that their self-attention mechanism, which will be covered in detail in chapter 3, doesn't have a notion of position or order for the tokens within a sequence.

The way the previously introduced embedding layer works is that the same token ID always gets mapped to the same vector representation, regardless of where the token ID is positioned in the input sequence, as illustrated in Figure 2.17.

Figure 2.17 The embedding layer converts a token ID into the same vector representation regardless of where it is located in the input sequence. For example, the token ID 5, whether it's in the first or third position in the token ID input vector, will result in the same embedding vector.



In principle, the deterministic, position-independent embedding of the token ID is good for reproducibility purposes. However, since the self-attention mechanism of LLMs itself is also position-agnostic, it is helpful to inject additional position information into the LLM.

To achieve this, there are two broad categories of position-aware embeddings: relative positional embeddings and absolute positional embeddings.

Absolute positional embeddings are directly associated with specific positions in a sequence. For each position in the input sequence, a unique embedding is added to the token's embedding to convey its exact location. For instance, the first token will have a specific positional embedding, the second token another distinct embedding, and so on, as illustrated in Figure 2.18.


Figure 2.18 Positional embeddings are added to the token embedding vector to create the input embeddings for an LLM. The positional vectors have the same dimension as the original token embeddings. The token embeddings are shown with value 1 for simplicity.



Instead of focusing on the absolute position of a token, the emphasis of relative positional embeddings is on the relative position or distance between tokens. This means the model learns the relationships in terms of "how far apart" rather than "at which exact position." The advantage here is that the model can generalize better to sequences of varying lengths, even if it hasn't seen such lengths during training.

Both types of positional embeddings aim to augment the capacity of LLMs to understand the order and relationships between tokens, ensuring more accurate and context-aware predictions. The choice between them often depends on the specific application and the nature of the data being processed.

OpenAI's GPT models use absolute positional embeddings that are optimized during the training process rather than being fixed or predefined like the positional encodings in the original Transformer model. This optimization process is part of the model training itself, which we will implement later in this book. For now, let's create the initial positional embeddings to create the LLM inputs for the upcoming chapters.

Previously, we focused on very small embedding sizes in this chapter for illustration purposes. We now consider more realistic and useful embedding sizes and encode the input tokens into a 256-dimensional vector representation. This is smaller than what the original GPT-3 model used (in GPT-3, the embedding size is 12,288 dimensions) but still reasonable for experimentation. Furthermore, we assume that the token IDs were created by the BPE tokenizer that we implemented earlier, which has a vocabulary size of 50,257:


output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)



Using the token_embedding_layer above, if we sample data from the data loader, we embed each token in each batch into a 256-dimensional vector. If we have a batch size of 8 with four tokens each, the result will be an 8 x 4 x 256 tensor.

Let's instantiate the data loader from section 2.6, Data sampling with a sliding window, first:


max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

he preceding code prints the following output:

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
 
Inputs shape:
 torch.Size([8, 4])



As we can see, the token ID tensor is 8x4-dimensional, meaning that the data batch consists of 8 text samples with 4 tokens each.

Let's now use the embedding layer to embed these token IDs into 256-dimensional vectors:



token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

The preceding print function call returns the following:


torch.Size([8, 4, 256])




As we can tell based on the 8x4x256-dimensional tensor output, each token ID is now embedded as a 256-dimensional vector.

For a GPT model's absolute embedding approach, we just need to create another embedding layer that has the same dimension as the token_embedding_layer:

context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_lengthe, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

 shown in the preceding code example, the input to the pos_embeddings is usually a placeholder vector torch.arange(context_length), which contains a sequence of numbers 0, 1, ..., up to the maximum input length − 1. The context_length is a variable that represents the supported input size of the LLM. Here, we choose it similar to the maximum length of the input text. In practice, input text can be longer than the supported context length, in which case we have to truncate the text.

The output of the print statement is as follows:


torch.Size([4, 256])


s we can see, the positional embedding tensor consists of four 256-dimensional vectors. We can now add these directly to the token embeddings, where PyTorch will add the 4x256-dimensional pos_embeddings tensor to each 4x256-dimensional token embedding tensor in each of the 8 batches:

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

The print output is as follows:

torch.Size([8, 4, 256])

The input_embeddings we created, as summarized in Figure 2.19, are the embedded input examples that can now be processed by the main LLM modules, which we will begin implementing in chapter 3

Figure 2.19 As part of the input processing pipeline, input text is first broken up into individual tokens. These tokens are then converted into token IDs using a vocabulary. The token IDs are converted into embedding vectors to which positional embeddings of a similar size are added, resulting in input embeddings that are used as input for the main LLM layers.



2.9 Summary

    LLMs require textual data to be converted into numerical vectors, known as embeddings since they can't process raw text. Embeddings transform discrete data (like words or images) into continuous vector spaces, making them compatible with neural network operations.
    As the first step, raw text is broken into tokens, which can be words or characters. Then, the tokens are converted into integer representations, termed token IDs.
    Special tokens, such as <|unk|> and <|endoftext|>, can be added to enhance the model's understanding and handle various contexts, such as unknown words or marking the boundary between unrelated texts.
    The byte pair encoding (BPE) tokenizer used for LLMs like GPT-2 and GPT-3 can efficiently handle unknown words by breaking them down into subword units or individual characters.
    We use a sliding window approach on tokenized data to generate input-target pairs for LLM training.
    Embedding layers in PyTorch function as a lookup operation, retrieving vectors corresponding to token IDs. The resulting embedding vectors provide continuous representations of tokens, which is crucial for training deep learning models like LLMs.
    While token embeddings provide consistent vector representations for each token, they lack a sense of the token's position in a sequence. To rectify this, two main types of positional embeddings exist: absolute and relative. OpenAI's GPT models utilize absolute positional embeddings that are added to the token embedding vectors and are optimized during the model training.
